In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess data
df = pd.read_csv('TimeSeries.csv')

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

In [ ]:
df.drop(['Year','Month'],axis=1,inplace=True)

In [ ]:
df.set_index('Date', inplace=True)
# remove the space before column name
df.rename(columns = {' Sales':'Sales'}, inplace = True)
df['Sales'] = df['Sales'].str.replace(',', '').astype(int) # Remove commas and convert to int

In [ ]:
df

In [ ]:
df.plot()

In [ ]:


from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
results = seasonal_decompose(df['Sales'])
results.plot();

In [ ]:
len(df)

In [ ]:
train = df.iloc[:141]
test = df.iloc[141:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [ ]:
df.head(),df.tail()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[:10]

In [ ]:

from keras.preprocessing.sequence import TimeseriesGenerator